In [15]:
import hashlib
%run SharedParams.ipynb

def get_merkle_root(data_list, index_start=None, index_end=None):
    if index_start == None:
        index_start = 0
    if index_end == None:
        index_end = len(data_list)
    assert math.log(index_end - index_start, 2) == int(math.log(index_end - index_start, 2))

    if index_end - index_start == 1:
        return hashlib.sha256(str(data_list[index_start]).encode("utf-8")).digest()
    new_data_list = []
    first_hash = get_merkle_root(data_list, index_start, (index_start + index_end)/2)
    second_hash = get_merkle_root(data_list, (index_start + index_end)/2, index_end)
    return hashlib.sha256(first_hash + second_hash).digest()


b'\xcdS\xa2\xceh\xe6Gl)Q.\xa5<9\\\x7f]\x8f\xbc\xb4aM\x89)\x8d\xb1N*[\xdbTV'


In [32]:
class MerkleTree:
    
    def __init__(self, data: list):
        self.tree_height = int(math.log(len(data), 2))
        assert self.tree_height == math.log(len(data), 2)
        
        self.data = data
        self.merkle = dict()
        
        for i in range(self.tree_height, 0, -1):
            for j in range(2**i):
                index = bin(j)[2:].rjust(i, "0")
                if i == self.tree_height:
                    self.merkle[index] = hashlib.sha256(str(data[j]).encode("utf-8")).digest()
                else:
                    self.merkle[index] = hashlib.sha256(self.merkle[index+"0"] + self.merkle[index+"1"]).digest()
        self.merkle[""] = hashlib.sha256(self.merkle["0"] + self.merkle["1"]).digest()
        self.root = self.merkle[""]
    
    def get_path(self, data_index):
        path = list()
        index = bin(data_index)[2:].rjust(self.tree_height, "0")
        while index != "":
            parent_index = index[:-1]
            neighbor_index = parent_index + "0" if parent_index == "1" else parent_index + "1"
            path.append(self.merkle[neighbor_index])
            index = parent_index
        return self.data[data_index], path
    
    @staticmethod
    def auth_path(data, path, root, tree_height):
        assert len(path) == tree_height
        curr_hash = hashlib.sha256(str(data).encode("utf-8")).digest()
        for h in path:
            curr_hash = hashlib.sha256(curr_hash + h).digest()
        
        assert curr_hash == root

{'00': b'k\x86\xb2s\xff4\xfc\xe1\x9dk\x80N\xffZ?WG\xad\xa4\xea\xa2/\x1dI\xc0\x1eR\xdd\xb7\x87[K', '01': b'\xd4s^:&^\x16\xee\xe0?Yq\x8b\x9b]\x03\x01\x9c\x07\xd8\xb6\xc5\x1f\x90\xda:fn\xec\x13\xab5', '10': b'N\x07@\x85b\xbe\xdb\x8b`\xce\x05\xc1\xde\xcf\xe3\xad\x16\xb7"0\x96}\xe0\x1fd\x0b~G)\xb4\x9f\xce', '11': b'K"ww\xd4\xdd\x1f\xc6\x1co\x88OHd\x1d\x02\xb4\xd1!\xd3\xfd2\x8c\xb0\x8bU1\xfc\xac\xda\xbf\x8a', '0': b'B\x95\xf7.\xeb\x1e5\x07\xb8F\x1e$\x0e;\x8d\x18\xc1\xe7\xbd/\x11"\xb1\x1f\xc9\xec@\xa6X\x94\x03\x1a', '1': b' \xabt}E\xa7y8\xa5\xb8L)D\xb8\xf55\\I\xf2\x1d\xb0\xc5IE\x1cb\x81\xc9\x1b\xa4\x8d\r', '': b'\xcdS\xa2\xceh\xe6Gl)Q.\xa5<9\\\x7f]\x8f\xbc\xb4aM\x89)\x8d\xb1N*[\xdbTV'}
[b'\xd4s^:&^\x16\xee\xe0?Yq\x8b\x9b]\x03\x01\x9c\x07\xd8\xb6\xc5\x1f\x90\xda:fn\xec\x13\xab5', b' \xabt}E\xa7y8\xa5\xb8L)D\xb8\xf55\\I\xf2\x1d\xb0\xc5IE\x1cb\x81\xc9\x1b\xa4\x8d\r']
success :)


In [29]:
hashlib.sha256(b"1").digest()

b'k\x86\xb2s\xff4\xfc\xe1\x9dk\x80N\xffZ?WG\xad\xa4\xea\xa2/\x1dI\xc0\x1eR\xdd\xb7\x87[K'